# Programming Python: The Short Story

This book demonstrates Python in action.

The task is to construct a database.

There are other things in Python not covered here.

# The Task

To keep a database of some people.

There are programing tools for this task, but the point is that we will learn a lot by programming it ourselves.

# Step 1: Representing Records

First we have to decide how to represent a single record.

## Using Lists

The lists can be used. 

In [5]:
bob = ['Bob Smith', 42, 30000, 'software']
sue = ['Sue Jones', 45, 40000, 'hardware']

Each record is a list.

In [7]:
bob[0], sue[2]

('Bob Smith', 40000)

Processing is easy:

In [9]:
bob[0].split()[-1]

'Smith'

In [10]:
sue[2] *= 1.25
sue

['Sue Jones', 45, 50000.0, 'hardware']

The processing for variable bob above is from the left to the right.

### Start-up pointers

We will work in jupyter notebook.

### A database list

First we combine variables bob and sue into a database.

In [14]:
people = [bob, sue]
for person in people:
    print(person)

['Bob Smith', 42, 30000, 'software']
['Sue Jones', 45, 50000.0, 'hardware']


The variable people is a database.

In [16]:
people[1][0]

'Sue Jones'

In [17]:
for person in people:
    print(person[0].split()[-1])
    person[2] *= 1.20

Smith
Jones


In [18]:
for person in people:
    print(person[2])

36000.0
60000.0


We can do list comprehension and maps.

In [20]:
pays = [person[2] for person in people]
pays

[36000.0, 60000.0]

In [21]:
pays = map(lambda x: x[2], people)
pays = list(pays)
pays

[36000.0, 60000.0]

In [22]:
sum(pays)

96000.0

In [23]:
sum(person[2] for person in people)

96000.0

Adding new records to the database

In [25]:
people.append(['Tom', 50, 0, None])

In [26]:
len(people)

3

In [27]:
for person in people:
    print(person)

['Bob Smith', 42, 36000.0, 'software']
['Sue Jones', 45, 60000.0, 'hardware']
['Tom', 50, 0, None]


In [28]:
people[-1][0]

'Tom'

Weaknes: our database is in memory only.

### Field labels

Another weaknes: we are accesing fields by integer positions.

Let's try to use the range function:

In [31]:
NAME, AGE, PAY = range(3)
bob = ['Bob Smith', 42, 10000]

In [32]:
bob[NAME]

'Bob Smith'

In [33]:
PAY, bob[PAY]

(2, 10000)

The uppercase variables have become field names.  But we remain dependant on this numbering.

Another problem: there is no mapping from the field instances names to field names.

We might try the list of tuples structure:

In [35]:
bob = [['name', 'Bob Smith'], ['age', 42], ['pay', 10000]]
sue = [['name', 'Sue Jones'], ['age', 45], ['pay', 20000]]
people = [bob, sue]
people

[[['name', 'Bob Smith'], ['age', 42], ['pay', 10000]],
 [['name', 'Sue Jones'], ['age', 45], ['pay', 20000]]]

But it stil does not solve the problem, since we still have to index by position:

In [37]:
for person in people:
    print(person[0][1], person[2][1])

Bob Smith 10000
Sue Jones 20000


In [38]:
[person[0][1] for person in people]

['Bob Smith', 'Sue Jones']

In [39]:
for person in people:
    print(person[0][1].split()[-1]) # give last names
    person[2][1] *= 1.10

Smith
Jones


In [40]:
for person in people:
    print(person[2])

['pay', 11000.0]
['pay', 22000.0]


Let us inspect field names in loops:

In [42]:
for person in people:
    for name, value in person:
        if name == 'name':
            print(value)

Bob Smith
Sue Jones


Better yet writing a *fetcher* function to do this:

In [44]:
def field(record, label):
    for fname, fvalue in record:
        if fname == label:
            return fvalue

In [45]:
bob

[['name', 'Bob Smith'], ['age', 42], ['pay', 11000.0]]

In [46]:
field(bob, 'name')

'Bob Smith'

In [47]:
field(sue, 'pay')

22000.0

In [48]:
for rec in people:
    print(field(rec, 'name'), field(rec, 'age'))

Bob Smith 42
Sue Jones 45


This leads to set of record interface functions.  In the next chapter we will find a better way.

### Using Dictionaries

Python dictionaries seem to be a natural solution:

In [51]:
bob = {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue = {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}

Now this is meaningful and does not depend on positions.

In [53]:
bob['name'], sue['pay']

('Bob Smith', 40000)

In [54]:
bob['name'].split()[-1]

'Smith'

In [55]:
sue['pay'] *= 1.1
sue['pay']

44000.0

Fields are accessed mnemonically now. This is more meaningful.

#### Other ways to make dictionaries

Namely there are several ways to do this.  Here is a function syntacs using named arguments (keyword arguments):

In [58]:
bob = dict(name='Bob Smith', age=42, pay=30000, job='dev')
sue = dict(name='Sue Jones', age=45, pay=40000, job='hdw')
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

In [59]:
sue

{'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}

alternativly we can fill individual fields in a dictionary:

In [61]:
sue = {}
sue['name'] = 'Sue Jones'
sue['age'] = 45
sue['pay'] = 40000
sue['job'] = 'htw'
sue

{'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'htw'}

or by the usage of the zip function:

In [63]:
names = ['name', 'age', 'pay', 'job']
values = ['Bob Smith', 42, 30000, 'dev']
list(zip(names, values))

[('name', 'Bob Smith'), ('age', 42), ('pay', 30000), ('job', 'dev')]

In [64]:
bob = dict(zip(names, values))
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

Finaly, we can initialize a dictionary like this:

In [66]:
fields = ('name', 'age', 'pay', 'job')

record = dict.fromkeys(fields, '?')

record

{'name': '?', 'age': '?', 'pay': '?', 'job': '?'}

#### Lists of dictionaries

We still need to colect records into a database:

In [68]:
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

In [69]:
sue

{'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'htw'}

In [142]:
people = [bob, sue]
people

[{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}]

In [144]:
for person in people:
    print(person['name'], person['pay'], sep=', ')

Bob Smith, 30000
Sue Jones, 40000


In [146]:
for person in people:
    if person['name'] == 'Sue Jones':
        print(person['pay'])

40000


We use keys here:

In [151]:
names = [person['name'] for person in people]
names

['Bob Smith', 'Sue Jones']

In [153]:
list(map(lambda x: x['name'], people))

['Bob Smith', 'Sue Jones']

In [155]:
sum(person['pay'] for person in people)

70000

Interestingly, the list comprehensions and the generator expressions can approach SQL queries:

In [158]:
[rec['name'] for rec in people if rec['age'] >= 45]

['Sue Jones']

In [ ]:
[rec['age'] ** 2 if rec['age'] >= 45 else rec['age']for rec in people if rec['age'] >= 45]

In [70]:
# initalize data to be stored in files, pickles, shelves

# records
bob = {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue = {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}
tom = {'name': 'Tom', 'age': 50, 'pay': 0, 'job': None}

# database
db = {}
db['bob'] = bob
db['sue'] = sue
db['tom'] = tom

if __name__ == '__main__':
    for key in db:
        print(key, '=>\n  ', db[key])


bob =>
   {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue =>
   {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}
tom =>
   {'name': 'Tom', 'age': 50, 'pay': 0, 'job': None}


# Shelves

In [72]:
from initdata import bob, sue
